# What is the MNIST Dataset

It is a dataset of images of handwritten digits. The dataset contains a training set of 60,000 examples, and a test set of 10,000 examples. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1.

The handwritten digits themselves were written by high school students and employees of the United States Census Bureau; divided into Special Database 1 and Special Database 3 respectively; this combination of two NIST databases are what form the MNIST dataset.

# Reading the MNIST Dataset 

In this section I'm going to be showing you how to read the MNIST dataset into memory from scratch. This is often done in one or two lines if using a library like Tensorflow or Keras eg.

```
from keras.datasets import mnist
# get the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
```

Pre-configured data files are fantastic and they save a lot of time, however it's important that we know how to prepare and load databases ourselves.

I tried to organize this Notebook into easy to digest sections:

1. Download and extract the four MNIST g-zipped files
2. Process the encoded images
3. Save the images to a dictionary
4. Store dictionary


# 1. Downloading and Extracting the Dataset

In [28]:
# Adapted from https://github.com/Ghosh4AI/Data-Processors/blob/master/MNIST/MNIST_Loader.ipynb

import os,urllib.request

# The path where the necessary files are downloaded to
datapath = './Data/MNISTData/'  

# Create the directory if it doesn't already exist
if not os.path.exists(datapath):
    os.makedirs(datapath)

# The necessary download URLS for the training and testing images/labels
urls = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
       'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
       'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
       'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

# Loop through each URL 
for url in urls:
    # GET FILENAME
    filename = url.split('/')[-1]
    
    if os.path.exists(datapath+filename):
        print(filename, ' already exists')  # CHECK IF FILE EXISTS
    else:
        print('Downloading ',filename)
        # Adapted from https://docs.python.org/3/howto/urllib2.html
        urllib.request.urlretrieve (url, datapath+filename)
print('All files are available')

All files are available


In [29]:
import os,gzip,shutil

# The path containing the MNIST data
datapath = './Data/MNISTData/'  

# List all folders in the directory
files = os.listdir(datapath)

for file in files:
    if file.endswith('gz'):
        print('Extracting ',file)
        with gzip.open(datapath + file, 'rb') as file_in:
            with open(datapath + file.split('.')[0], 'wb') as file_out:
                shutil.copyfileobj(file_in, file_out)
print('Extraction Complete\n')

# Clean up and remove the gz folders, run twice to remove all files
for file in files:
    print('Removing ', file)
    os.remove(datapath + file)
print ('All archives removed')

Extracting  t10k-images-idx3-ubyte.gz
Extracting  train-images-idx3-ubyte.gz
Extracting  train-labels-idx1-ubyte.gz
Extracting  t10k-labels-idx1-ubyte.gz
Extraction Complete

Removing  t10k-images-idx3-ubyte.gz
Removing  train-images-idx3-ubyte.gz
Removing  train-labels-idx1-ubyte.gz
Removing  t10k-labels-idx1-ubyte.gz
All archives removed
